In [ ]:
#Decision tree

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import GridSearchCV


In [2]:
df = pd.read_csv("/Users/kimhyunmin/vehicle_data.csv", low_memory=False)

In [3]:
df.shape

(101174, 19)

EDA

In [3]:
import pandas_profiling

pr=df.profile_report() 
df.profile_report()
pr.to_file('/Users/kimhyunmin/vehicle_data.html')

variable selection

In [14]:
df.drop(df.columns[[0,1,13,14,15,16,17,18]], axis=1, inplace=True)

In [15]:
df.columns

Index(['Price', 'Brand', 'Model', 'Edition', 'Year', 'Condition',
       'Transmission', 'Body', 'Fuel', 'Capacity', 'Mileage'],
      dtype='object')

In [16]:
df = df.dropna(axis=0)

In [17]:
df.shape

(27499, 11)

In [18]:
df['Price'] = df['Price'].str.replace('Rs', '').str.replace(',', '').astype('float')
df['Capacity'] = df['Capacity'].str.replace('cc', '').str.replace(',', '').astype('int64')
df['Mileage'] = df['Mileage'].str.replace('km', '').str.replace(',', '').astype('int64')

In [19]:
import pandas_profiling

pr=df.profile_report() 
df.profile_report()
pr.to_file('/Users/kimhyunmin/vehicle_data2.html')

One-hot 인코딩

In [15]:
df = pd.get_dummies(df, columns=["Brand", "Model", "Edition", "Condition", "Transmission", "Body", "Fuel"], drop_first=True)

In [16]:
X = df.iloc[:, 1:]
y = df.iloc[:, :1]

scaling

In [17]:
from sklearn.preprocessing import MinMaxScaler

y.sort_index(ascending=False).reset_index(drop=True)

yscaler = MinMaxScaler()

y = yscaler.fit_transform(y)
y = pd.DataFrame(y)

In [18]:
X.sort_index(ascending=False).reset_index(drop=True)

Xscaler = MinMaxScaler()

X = Xscaler.fit_transform(X)
X = pd.DataFrame(X)

split

In [19]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.3, random_state=2021)
print(len(train), len(test))

19249 8250


In [20]:
trainX, testX = X[0:19249], X[19249:]
trainy, testy = y[0:19249], y[19249:]
print(len(trainX), len(testX), len(trainy), len(testy))

19249 8250 19249 8250


In [21]:
testy = yscaler.inverse_transform(testy)
testy

array([[18200000.],
       [ 4375000.],
       [ 1675000.],
       ...,
       [ 2940000.],
       [ 8850000.],
       [ 8150000.]])

In [22]:
def performance_metric(y_true, y_predict):
    error = metrics.mean_squared_error(y_true, y_predict)
    return error

In [23]:
def fit_model(data, target):
    regressor = DecisionTreeRegressor()
    param_grid = {'max_depth':(5,10,15,20,25,30)}
    scoring_fnc = metrics.make_scorer(performance_metric, False)
    reg = GridSearchCV(regressor, param_grid, scoring = scoring_fnc, cv = 3)
    reg.fit(data, target)
    return reg.best_estimator_

In [24]:
regression = fit_model(trainX, trainy)
regression

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass greater_is_better=False as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


DecisionTreeRegressor(max_depth=15)

In [25]:
dtReg = DecisionTreeRegressor(max_depth=15)
dtReg = dtReg.fit(trainX, trainy)

In [26]:
predY = dtReg.predict(testX)
predY

array([0.09285066, 0.02082634, 0.01943122, ..., 0.01197584, 0.03738979,
       0.03738979])

Prediction

In [27]:
predY = yscaler.inverse_transform(predY.reshape(-1,1))
predY

array([[19500000.        ],
       [ 4375000.        ],
       [ 4082027.02702703],
       ...,
       [ 2516408.16326531],
       [ 7853298.84482758],
       [ 7853298.84482758]])

RMSE

In [28]:
from sklearn.metrics import mean_squared_error
def RMSE(testy, predY):
    return np.sqrt(mean_squared_error(testy, predY))
print('RMSE : ', RMSE(testy, predY))  

RMSE :  3558386.5270222253


In [ ]:
#SVR

In [32]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

param_grid = [ {'kernel': ['linear'], 'C': [10,30,100]},
                {'kernel': ['rbf'], 'C': [10,30,100],
                        'gamma': [0.01,0.03,0.1],}
]

svm_reg=SVR()

scoring_fnc = metrics.make_scorer(performance_metric, False)

grid_search = GridSearchCV(svm_reg, param_grid, cv=5, scoring=scoring_fnc, verbose=2)
grid_search.fit(trainX, trainy)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass greater_is_better=False as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] C=10, kernel=linear .............................................


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[CV] .............................. C=10, kernel=linear, total= 1.8min
[CV] C=10, kernel=linear .............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.8min remaining:    0.0s
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


...

[CV] ..................... C=100, gamma=0.1, kernel=rbf, total= 1.8min
[CV] C=100, gamma=0.1, kernel=rbf ....................................


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[CV] ..................... C=100, gamma=0.1, kernel=rbf, total= 1.8min


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed: 124.1min finished
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GridSearchCV(cv=5, estimator=SVR(),
             param_grid=[{'C': [10, 30, 100], 'kernel': ['linear']},
                         {'C': [10, 30, 100], 'gamma': [0.01, 0.03, 0.1],
                          'kernel': ['rbf']}],
             scoring=make_scorer(performance_metric, greater_is_better=False),
             verbose=2)

In [34]:
grid_search.fit(trainX, trainy)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SVR()

In [38]:
predy = grid_search.predict(testX)
predy

array([0.16871017, 0.10274433, 0.01424655, ..., 0.00535266, 0.11378735,
       0.07098807])

prediction

In [39]:
predy = yscaler.inverse_transform(predy.reshape(-1,1))
predy

array([[35430382.82223117],
       [21577655.83278765],
       [ 2993254.36470748],
       ...,
       [ 1125550.2122298 ],
       [23896673.10683026],
       [14908888.73869616]])

RMSE

In [40]:
print('RMSE : ', RMSE(testy, predy))  

RMSE :  9831993.865704942
